# Merging Data

Combining and reshaping data from multiple sources

For this module, we will be exploring the techniques for combining and reshaping data from multiple sources. You will learn about:

* Combining data using pandas library
* Validating Merges
* Debugging Chains
* Exporting to Excel


In [1]:
import numpy as np
import pandas as pd
import polars as pl
import zipfile

In [2]:
# https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data?resource=download
with zipfile.ZipFile('data/AB_NYC_2019.csv.zip') as zip:
  print(zip.namelist())

['AB_NYC_2019.csv']


In [3]:
ab = pd.read_csv('data/AB_NYC_2019.csv.zip',
                 dtype_backend='pyarrow',
                 engine='pyarrow')
ab.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,<NA>,<NA>,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.1,1,0


In [4]:
ab_pl = pl.read_csv(zipfile.ZipFile("data/AB_NYC_2019.csv.zip").read("AB_NYC_2019.csv"))

In [5]:
temps = pd.read_csv('data/nyc-ab-temp.csv',
                    index_col=0,
                    dtype_backend='pyarrow',
                    engine='pyarrow')

In [6]:
temps_pl = (pl
            .scan_csv("data/nyc-ab-temp.csv")
            .drop("")
            .collect()
            )

In [7]:
temps.head()

,lat,lon,temp
,,,
0,40.64749,-73.97237,72.0
1,40.75362,-73.98377,57.0
2,40.80902,-73.94190,76.0
3,40.68514,-73.95976,24.0
4,40.79851,-73.94399,27.0


In [8]:
temps_pl.head()

lat,lon,temp
f64,f64,f64
40.64749,-73.97237,72.0
40.75362,-73.98377,57.0
40.80902,-73.9419,76.0
40.68514,-73.95976,24.0
40.79851,-73.94399,27.0


## Merging

In [9]:
# Error expected here. We're working to fix it in the next few cells.
# (ab
#  .merge(temps)
# )

In [10]:
(ab.columns.intersection(temps.columns))

Index([], dtype='object')

In [11]:
(set(ab_pl.columns).intersection(temps_pl.columns))

set()

In [12]:
ab.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [13]:
ab_pl.columns

['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [14]:
temps.columns

Index(['lat', 'lon', 'temp'], dtype='object')

In [15]:
temps_pl.columns

['lat', 'lon', 'temp']

In [16]:
temps = temps.rename(columns=dict(lat="latitude", lon="longitude"))

In [17]:
(ab
 .merge(temps, on=["latitude", "longitude"])
 .sample(10)
)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,temp
11269,8733620,Lg private room minutes from subway,3571821,Andy & Friends,Brooklyn,Bedford-Stuyvesant,40.69053,-73.95632,Private room,45,5,32,2017-12-26,0.7,4,0,86.0
25973,20738759,Sun-Filled Cobble Hill Loft,2687071,Lily,Brooklyn,Brooklyn Heights,40.69040,-73.99376,Entire home/apt,135,3,21,2019-05-05,0.95,1,15,48.0
31467,24477757,Large Private Room in a Magnificent Penthouse ...,169382341,Tomás,Manhattan,East Village,40.72200,-73.98396,Private room,89,5,11,2019-06-27,0.84,2,76,78.0
7068,5091056,Comfy Bedroom quite neighborhood,26301893,Claude,Queens,Forest Hills,40.71158,-73.85273,Private room,80,1,12,2019-01-03,0.35,1,59,58.0
3322,1983188,1BR near Columbia / Central Park,1521508,Jose,Manhattan,Upper West Side,40.79859,-73.96078,Private room,80,3,38,2019-06-26,0.59,1,344,71.0
33116,26126570,Two bedroom apartment with all amenities,673215,Jessica Rose,Manhattan,Harlem,40.82299,-73.95461,Entire home/apt,200,5,0,<NA>,<NA>,2,39,37.0
5489,3970701,Beautiful Private Room in Newly Renovated Apt,20278196,Stephanie,Brooklyn,East Flatbush,40.66303,-73.92526,Private room,60,2,6,2019-02-03,0.26,3,326,71.0
12510,9613541,Open Room for December,49738780,Katie,Queens,Astoria,40.77211,-73.92055,Private room,100,1,0,<NA>,<NA>,1,0,76.0
47582,35809258,Cozy and Modern 3BR for upto 6 ppl - 4min to m...,269220629,Isidro M.,Brooklyn,Crown Heights,40.67671,-73.93991,Entire home/apt,169,4,0,<NA>,<NA>,1,324,50.0
30455,23559016,Lovely room in heart of Williamsburg,173021064,Katie,Brooklyn,Williamsburg,40.71081,-73.94965,Private room,75,2,2,2018-04-22,0.13,2,7,48.0


In [18]:
temps_pl = temps_pl.rename(dict(lat="latitude", lon="longitude"))

In [19]:
(ab_pl
 .join(temps_pl, on=["latitude", "longitude"])
 .sample(10)
 )

id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,temp
i64,str,i64,str,str,str,f64,f64,str,i64,i64,i64,str,f64,i64,i64,f64
15016999,"""Crown Heights …",94782931,"""Joe""","""Brooklyn""","""Crown Heights""",40.66692,-73.95205,"""Entire home/ap…",105,1,7,"""2019-06-19""",0.21,1,6,76.0
18066733,"""5br Duplex. @ …",117492425,"""Dine""","""Staten Island""","""St. George""",40.64605,-74.07897,"""Entire home/ap…",289,4,2,"""2018-07-25""",0.16,6,230,27.0
1439162,"""Cozy & Private…",7460181,"""Geraldine""","""Brooklyn""","""Bedford-Stuyve…",40.68238,-73.93795,"""Entire home/ap…",99,3,208,"""2019-06-25""",2.89,1,206,29.0
167013,"""Spacious moder…",306605,"""Daniel""","""Manhattan""","""Chelsea""",40.74342,-73.99483,"""Entire home/ap…",205,9,3,"""2018-05-13""",0.04,2,76,43.0
6862600,"""Bright Parksid…",21705761,"""Jonathan""","""Brooklyn""","""Park Slope""",40.66722,-73.97811,"""Entire home/ap…",215,2,2,"""2016-04-05""",0.05,1,0,74.0
13884778,"""Luxury Room (r…",21410914,"""Eugene""","""Manhattan""","""Morningside He…",40.81093,-73.95928,"""Private room""",100,3,81,"""2019-06-22""",2.24,6,133,70.0
24159289,"""Great View of …",41366374,"""Jiangyang""","""Queens""","""Long Island Ci…",40.75047,-73.9408,"""Entire home/ap…",150,1,0,null,null,1,0,28.0
19550594,"""Williamsburg O…",19631496,"""The""","""Brooklyn""","""Williamsburg""",40.70706,-73.94428,"""Entire home/ap…",225,5,75,"""2019-06-17""",3.49,2,278,52.0
11032957,"""Glorious brown…",57254931,"""Courtney""","""Brooklyn""","""Clinton Hill""",40.68121,-73.96319,"""Entire home/ap…",190,7,60,"""2019-02-17""",1.66,1,0,75.0


## Validating Merges

In [20]:
left = pd.DataFrame({'name': ['Ravi', 'Jose', 'Jose'],
                    'pet': ['Dog', 'Cat', 'Dog']})
right = pd.DataFrame({'name': ['Ravi', 'Jose', 'Sally'],
                      'age': [10, 17, 5]})

In [21]:
display(left)
display(right)

,name,pet
0,Ravi,Dog
1,Jose,Cat
2,Jose,Dog


,name,age
0,Ravi,10
1,Jose,17
2,Sally,5


In [22]:
left.merge(right, how='inner')

,name,pet,age
0,Ravi,Dog,10
1,Jose,Cat,17
2,Jose,Dog,17


In [23]:
left.merge(right, how='left')

,name,pet,age
0,Ravi,Dog,10
1,Jose,Cat,17
2,Jose,Dog,17


In [24]:
left.merge(right, how='right')

,name,pet,age
0,Ravi,Dog,10
1,Jose,Cat,17
2,Jose,Dog,17
3,Sally,NaN,5


In [25]:
left.merge(right, how='outer')

,name,pet,age
0,Ravi,Dog,10
1,Jose,Cat,17
2,Jose,Dog,17
3,Sally,NaN,5


In [26]:
left.merge(right, how='cross')

,name_x,pet,name_y,age
0,Ravi,Dog,Ravi,10
1,Ravi,Dog,Jose,17
2,Ravi,Dog,Sally,5
3,Jose,Cat,Ravi,10
4,Jose,Cat,Jose,17
5,Jose,Cat,Sally,5
6,Jose,Dog,Ravi,10
7,Jose,Dog,Jose,17
8,Jose,Dog,Sally,5


In [27]:
left

,name,pet
0,Ravi,Dog
1,Jose,Cat
2,Jose,Dog


In [28]:
# left.merge(right, how='inner', validate='1:1')

In [29]:
left.merge(right, how='inner', validate='m:1')

,name,pet,age
0,Ravi,Dog,10
1,Jose,Cat,17
2,Jose,Dog,17


## Debugging Trick

In [30]:
def limit(df:pd.DataFrame, n_rows:int, n_cols:int) -> pd.DataFrame:
  return df.iloc[:n_rows, :n_cols]

(ab
 .pipe(limit, n_rows=3, n_cols=6)
)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem


In [31]:
def limit(df:pl.DataFrame, n_rows:int, n_cols:int):
    return df[:n_rows, :n_cols]


(ab_pl
 .pipe(limit, n_rows=3, n_cols=6)
)

id,name,host_id,host_name,neighbourhood_group,neighbourhood
i64,str,i64,str,str,str
2539,"""Clean & quiet …",2787,"""John""","""Brooklyn""","""Kensington"""
2595,"""Skylit Midtown…",2845,"""Jennifer""","""Manhattan""","""Midtown"""
3647,"""THE VILLAGE OF…",4632,"""Elisabeth""","""Manhattan""","""Harlem"""


In [32]:
# add debug to helpers
def debug(df:pd.DataFrame, extra:str='') -> pd.DataFrame:
  print(f'{extra} {df.shape=}')
  return df

(ab
 .pipe(debug, extra='before')
 .merge(temps, on=['latitude', 'longitude'])
 .pipe(debug, extra='after') 
)

before df.shape=(48895, 16)
after df.shape=(48895, 17)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,temp
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,72.0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,57.0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,<NA>,<NA>,1,365,76.0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,24.0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.1,1,0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,<NA>,<NA>,2,9,76.0
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,<NA>,<NA>,2,36,71.0
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,<NA>,<NA>,1,27,44.0
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,<NA>,<NA>,6,2,73.0


In [33]:
def debug(df:pl.DataFrame, extra:str=""):
    print(f"{extra} {df.shape=}")
    return df


(ab_pl
 .pipe(debug, extra="before")
 .join(temps_pl, on=["latitude", "longitude"])
 .pipe(debug, extra="after")
)

before df.shape=(48895, 16)
after df.shape=(48895, 17)


id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,temp
i64,str,i64,str,str,str,f64,f64,str,i64,i64,i64,str,f64,i64,i64,f64
2539,"""Clean & quiet …",2787,"""John""","""Brooklyn""","""Kensington""",40.64749,-73.97237,"""Private room""",149,1,9,"""2018-10-19""",0.21,6,365,72.0
2595,"""Skylit Midtown…",2845,"""Jennifer""","""Manhattan""","""Midtown""",40.75362,-73.98377,"""Entire home/ap…",225,1,45,"""2019-05-21""",0.38,2,355,57.0
3647,"""THE VILLAGE OF…",4632,"""Elisabeth""","""Manhattan""","""Harlem""",40.80902,-73.9419,"""Private room""",150,3,0,null,null,1,365,76.0
3831,"""Cozy Entire Fl…",4869,"""LisaRoxanne""","""Brooklyn""","""Clinton Hill""",40.68514,-73.95976,"""Entire home/ap…",89,1,270,"""2019-07-05""",4.64,1,194,24.0
5022,"""Entire Apt: Sp…",7192,"""Laura""","""Manhattan""","""East Harlem""",40.79851,-73.94399,"""Entire home/ap…",80,10,9,"""2018-11-19""",0.1,1,0,27.0
5099,"""Large Cozy 1 B…",7322,"""Chris""","""Manhattan""","""Murray Hill""",40.74767,-73.975,"""Entire home/ap…",200,3,74,"""2019-06-22""",0.59,1,129,72.0
5121,"""BlissArtsSpace…",7356,"""Garon""","""Brooklyn""","""Bedford-Stuyve…",40.68688,-73.95596,"""Private room""",60,45,49,"""2017-10-05""",0.4,1,0,81.0
5178,"""Large Furnishe…",8967,"""Shunichi""","""Manhattan""","""Hell's Kitchen…",40.76489,-73.98493,"""Private room""",79,2,430,"""2019-06-24""",3.47,1,220,39.0
5203,"""Cozy Clean Gue…",7490,"""MaryEllen""","""Manhattan""","""Upper West Sid…",40.80178,-73.96723,"""Private room""",79,2,118,"""2017-07-21""",0.99,1,0,42.0


In [34]:
(ab
 .pipe(debug, extra='before')
 .merge(temps, on=['latitude', 'longitude'])
 .pipe(debug, extra='after') 
 .groupby('neighbourhood_group')
 .mean(numeric_only = True)
 .pipe(debug, extra='summary') 
)

before df.shape=(48895, 16)
after df.shape=(48895, 17)
summary df.shape=(5, 11)


,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,temp
neighbourhood_group,,,,,,,,,,,
Bronx,2.273492e+07,1.056099e+08,40.848305,-73.884552,87.496792,4.560953,26.004583,1.837831,2.233731,165.758937,55.450459
Brooklyn,1.825685e+07,5.671526e+07,40.685036,-73.951190,124.383207,6.056556,24.202845,1.283212,2.284371,100.232292,56.169537
Manhattan,1.877494e+07,6.783062e+07,40.765062,-73.974607,196.875814,8.579151,20.985596,1.272131,12.791330,111.979410,55.908629
Queens,2.175500e+07,9.615680e+07,40.731531,-73.872775,99.517649,5.181433,27.700318,1.941200,4.060184,144.451818,55.913604
Staten Island,2.159747e+07,9.853360e+07,40.610225,-74.105805,114.812332,4.831099,30.941019,1.872580,2.319035,199.678284,55.873995


In [35]:
(ab_pl
 .pipe(debug, extra="before")
 .join(temps_pl, on=["latitude", "longitude"])
 .pipe(debug, extra="after")
 .group_by("neighbourhood_group")
 .mean()
 .select(pl.col(pl.Float64))
 .pipe(debug, extra="summary")
)

before df.shape=(48895, 16)
after df.shape=(48895, 17)
summary df.shape=(5, 11)


id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,temp
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2.2735e7,1.0561e8,40.848305,-73.884552,87.496792,4.560953,26.004583,1.837831,2.233731,165.758937,55.450459
2.1597e7,9.8534e7,40.610225,-74.105805,114.812332,4.831099,30.941019,1.87258,2.319035,199.678284,55.873995
2.1755e7,9.6157e7,40.731531,-73.872775,99.517649,5.181433,27.700318,1.9412,4.060184,144.451818,55.913604
1.8257e7,5.6715e7,40.685036,-73.95119,124.383207,6.056556,24.202845,1.283212,2.284371,100.232292,56.169537
1.8775e7,6.7831e7,40.765062,-73.974607,196.875814,8.579151,20.985596,1.272131,12.79133,111.97941,55.908629


## Cleanup Columns

In [36]:
(ab
 .pipe(debug, extra='before')
 .merge(temps, on=['latitude', 'longitude']
       )
 .pipe(debug, extra='after') 
 .columns
)

before df.shape=(48895, 16)
after df.shape=(48895, 17)


Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'temp'],
      dtype='object')

In [37]:
(ab_pl
 .pipe(debug, extra="before")
 .join(temps_pl, on=["latitude", "longitude"])
 .pipe(debug, extra="after")
 .columns
)

before df.shape=(48895, 16)
after df.shape=(48895, 17)


['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'temp']

In [38]:
# Explicitly list columns
(ab
 .pipe(debug, extra='before')
 .merge(temps, on=['latitude', 'longitude'])
 .pipe(debug, extra='after') 
 .loc[:, ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'temp']]
 .pipe(debug, extra='limit cols')  
)

before df.shape=(48895, 16)
after df.shape=(48895, 17)
limit cols df.shape=(48895, 17)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,temp
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,72.0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,57.0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,<NA>,<NA>,1,365,76.0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,24.0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.1,1,0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,<NA>,<NA>,2,9,76.0
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,<NA>,<NA>,2,36,71.0
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,<NA>,<NA>,1,27,44.0
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,<NA>,<NA>,6,2,73.0


In [39]:
# Explicitly list columns
(ab_pl
 .pipe(debug, extra="before")
 .join(temps_pl, on=["latitude", "longitude"])
 .pipe(debug, extra="after")
 [:, [
        "id",
        "name",
        "host_id",
        "host_name",
        "neighbourhood_group",
        "neighbourhood",
        "latitude",
        "longitude",
        "room_type",
        "price",
        "minimum_nights",
        "number_of_reviews",
        "last_review",
        "reviews_per_month",
        "calculated_host_listings_count",
        "availability_365",
        "temp",
        ],
    ]
 .pipe(debug, extra="limit cols")
)

before df.shape=(48895, 16)
after df.shape=(48895, 17)
limit cols df.shape=(48895, 17)


id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,temp
i64,str,i64,str,str,str,f64,f64,str,i64,i64,i64,str,f64,i64,i64,f64
2539,"""Clean & quiet …",2787,"""John""","""Brooklyn""","""Kensington""",40.64749,-73.97237,"""Private room""",149,1,9,"""2018-10-19""",0.21,6,365,72.0
2595,"""Skylit Midtown…",2845,"""Jennifer""","""Manhattan""","""Midtown""",40.75362,-73.98377,"""Entire home/ap…",225,1,45,"""2019-05-21""",0.38,2,355,57.0
3647,"""THE VILLAGE OF…",4632,"""Elisabeth""","""Manhattan""","""Harlem""",40.80902,-73.9419,"""Private room""",150,3,0,null,null,1,365,76.0
3831,"""Cozy Entire Fl…",4869,"""LisaRoxanne""","""Brooklyn""","""Clinton Hill""",40.68514,-73.95976,"""Entire home/ap…",89,1,270,"""2019-07-05""",4.64,1,194,24.0
5022,"""Entire Apt: Sp…",7192,"""Laura""","""Manhattan""","""East Harlem""",40.79851,-73.94399,"""Entire home/ap…",80,10,9,"""2018-11-19""",0.1,1,0,27.0
5099,"""Large Cozy 1 B…",7322,"""Chris""","""Manhattan""","""Murray Hill""",40.74767,-73.975,"""Entire home/ap…",200,3,74,"""2019-06-22""",0.59,1,129,72.0
5121,"""BlissArtsSpace…",7356,"""Garon""","""Brooklyn""","""Bedford-Stuyve…",40.68688,-73.95596,"""Private room""",60,45,49,"""2017-10-05""",0.4,1,0,81.0
5178,"""Large Furnishe…",8967,"""Shunichi""","""Manhattan""","""Hell's Kitchen…",40.76489,-73.98493,"""Private room""",79,2,430,"""2019-06-24""",3.47,1,220,39.0
5203,"""Cozy Clean Gue…",7490,"""MaryEllen""","""Manhattan""","""Upper West Sid…",40.80178,-73.96723,"""Private room""",79,2,118,"""2017-07-21""",0.99,1,0,42.0


## Export to Excel

In [40]:
with pd.ExcelWriter("data/airbnb.xlsx") as xls_out:
      xl = (ab
      .pipe(debug, extra='before')
      .merge(temps, on=['latitude', 'longitude'])
      .pipe(debug, extra='after') 
      .loc[:, ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
            'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
            'minimum_nights', 'number_of_reviews', 'last_review',
            'reviews_per_month', 'calculated_host_listings_count',
            'availability_365', 'temp']]
      )

      xl.to_excel(xls_out, sheet_name='all')
      (xl
      .query('neighbourhood_group=="Brooklyn"')
      .to_excel(xls_out, sheet_name='Brooklyn')
      )

before df.shape=(48895, 16)
after df.shape=(48895, 17)


In [41]:
from xlsxwriter import Workbook

with Workbook("data/airbnb_polars.xlsx") as xls_out:
      xl = (ab_pl
            .pipe(debug, extra='before')
            .join(temps_pl, on=['latitude', 'longitude'])
            .pipe(debug, extra='after') 
            [:, ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
                  'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
                  'minimum_nights', 'number_of_reviews', 'last_review',
                  'reviews_per_month', 'calculated_host_listings_count',
                  'availability_365', 'temp']]
            )

      xl.write_excel(xls_out, worksheet='all')
      (xl
      .filter(pl.col('neighbourhood_group') == "Brooklyn")
      .write_excel(xls_out, worksheet='Brooklyn')
      )

before df.shape=(48895, 16)
after df.shape=(48895, 17)
